In [1]:
%autosave 3600

Autosaving every 3600 seconds


# Erste Schritte mit dem Modul `fluids`

Das Modul `fluids` kann genutzt werden, um Zustandsgrößen von Fluiden, wie Luft, Wasser oder Kältemitteln wie R134a, R290, zu ermitteln. 

Weiter besteht die Möglichkeit, die Zustandsgrößen feuchter Luft zu ermitteln.

Fast alle Berechnungen in `fluids` werden mit Hilfe von `CoolProp` durchgeführt. Das Modul `fluids` stellt lediglich einen Wrapper zur Verfügung, der in Jupyter Notebooks einen vereinfachten Aufruf der Funktionen `CoolProps.CoolProps.HAPropsSi` (für feuchte Luft) und `CoolProps.CoolProps.PropsSi` für andere Fluide zur Verfügung stellt.

Von `fluids` werden die Objekte `fluid_factory`, `Q_` (oder gleichwertig `Quantity`), `T_0` sowie `p_amb` zur Verfügung gestellt. Sie werden wie üblich in Pythonprogrammen oder Jupyter Notebooks importiert:

In [2]:
from fluids import fluid_factory, Q_, T_0, p_amb

## Die Größen `T_0` und `p_amb`

- Die Größe `T_0` ist der Zahlenwert der Temperatur 
$T_0 = 273.15\,\mathrm{K} = 0\,°\mathrm{C}$
in $\mathrm{K}$.
- Die Größe `p_amb` ist der Zahlenwert des Luftdrucks 
$p_\text{amb} = 101325\,\mathrm{Pa}$ 
in $\mathrm{Pa}$

In [3]:
(T_0, p_amb)

(273.15, 101325)

## Das Objekt `Q_`

Das Objekt `Q_` (oder gleichwertig `Quantity`) stammt aus dem Modul `pint` und ermöglicht das Arbeiten mit einheitenbehafteten Größen. Einen guten Überblick über die Möglichkeiten von `pint` findet man unter

https://pint.readthedocs.io/en/stable/

Als Beispiel werden die Strecke $s = 120\,\mathrm{km}$ und die Zeit $t = 2\,\mathrm{h}$ als Größen dargestellt:

In [4]:
s = Q_(120,'km')
s

120 <Unit('kilometer')>

In [5]:
t = Q_(2,'hour') # Achtung: hour, nicht h!
t

2 <Unit('hour')>

Beim Aufruf einer Größe ist ein Zahlenwert und eine Einheit anzugeben. Mit einheitenbehafteten Größen kann gerechnet werden. Wird die oben definierte Strecke $s$ in der Zeit $t$ zurückgelegt, so ist die Geschwindigkeit

$$
  v = \frac{s}{t}
$$

In [6]:
v = s/t
v

60.0 <Unit('kilometer / hour')>

Innerhalb des Moduls `fluids` sind für das Objekt `Q_` die zusätzlichen Properties

`Q_.base_value = Q_.bv`

definiert worden. Damit lässt sich der Zahlenwert einer Größe in Grundeinheiten angeben:

In [7]:
v.to_base_units()

16.666666666666668 <Unit('meter / second')>

In [8]:
v.bv

16.666666666666668

In den Anwendungen der Heizungs-, Lüftungs- und Klimatechnik werden häufig die Einheiten Prozent und ppM (part per Million) benutzt. Diese wurden zusätzlich definiert:

In [9]:
eta = Q_(80,'percent')
eta

80 <Unit('percent')>

In [10]:
eta.bv

0.8

In [11]:
k = Q_(300,'ppM')
k

300 <Unit('ppM')>

In [12]:
k.bv

0.0003

Für eine intensivere Beschäftigung mit den Möglichkeiten von `pint` und den daraus importierten Objekten wird auf die oben genannte Dokumentation verwiesen.

## Das Objekt `fluid_factory`

Für das Objekt `fluid_factory` existieren drei Aufrufparameter:

`Klasse = fluid_factory(Stoff, with_units=False, p_default=p_amb)`

Der erste Parameter ist obligatorisch und definiert den Stoff, für den diese Klasse gilt.

Zulässig sind alle Stoffe, die auf der Seite

http://www.coolprop.org/coolprop/HighLevelAPI.html#table-of-string-inputs-to-propssi-function

aufgeführt sind, sowie `Stoff='HumidAir'` für feuchte Luft. Für alle Stoffe außer `HumidAir` werden die konkreten Berechnungen durch die Funktion

`CoolProp.CoolProp.PropsSi(...)`

durchgeführt, für `HumidAir` wird die Funktion

`CoolProp.CoolProp.HAPropsSi(...)`

benutzt.

Der Parameter `p_default=Druck` ist nur für feuchte Luft (`Stoff='HumidAir'`) definiert.

Wird mit einem Fluid, zum Beispiel R134a gearbeitet, so lautet der Aufruf

In [13]:
R_134a = fluid_factory('R134a')

# oder
R_134a_wu = fluid_factory('R134a',with_units=True)

Sind zwei unabhängige Zustandsgrößen bekannt, so lassen sich die weiteren Zustandsgrößen berechnen. Die folgenden Zustandsgrößen werden von `fluids` standardmäßig berücksichtigt:

In [14]:
R_134a.acceptable_args

{'P': 'Pa',
 'Pcrit': 'Pa',
 'T': 'K',
 'Tcrit': 'K',
 'D': 'kg/m**3',
 'H': 'J/kg',
 'U': 'J/kg',
 'S': 'J/kg/K',
 'A': 'm/s',
 'L': 'W/m/K',
 'M': 'kg/mol',
 'C': 'J/kg/K',
 'CVMASS': 'J/kg/K',
 'Q': '',
 'Z': '',
 'V': 'Pa*s'}


Größe    | Einheit  | Bedeutung
---------|----------|----------
P        | Pa       | Druck
Pcrit    | Pa       | kritischer Druk
T        | K        | Temperatur
Tcrit    | K        | kritische Temperatur
D        | kg/m**3  | Dichte
H        | J/kg     | spezifische Enthalpie
U        | J/kg     | spezifische innere Energie
S        | J/kg/K   | spezifische Entropie
A        | m/s      | Schallgeschwindigkeit
L        | W/m/K    | Wärmeleitfähigkeit
M        | kg/mol   | Molmasse
C        | J/kg/K   | spezifische Wärmekapazität bei p=const
CVMASS   | J/kg/K   | spezifische Wärmekapazität bei v=const
Q        |          | Dampfanteil im Nassdampf oder -1
Z        |          | Kompressibilität 
V        | Pa*s     | Viskosität

Durch `P = 3bar`, `Q=1` ist ein Zustandspunkt bei einem Druck von 3bar bei 100% Sattdampf festgelegt. Dieser Punkt wird durch den Aufruf

In [15]:
p_1 = R_134a(
    P=3e5, # Druck in Pa
    Q=1, # 100 % Dampf 
    name='P_1', # Bezeichnung
)

festgelegt. Sobald er festgelegt ist, lassen sich die oben aufgeführten Zustandswerte abrufen:

In [16]:
p_1.args

{'P': 300000.0,
 'Pcrit': 4059280.0,
 'T': 273.8220637378023,
 'Tcrit': 374.21,
 'D': 14.77016899135526,
 'H': 398995.14983938605,
 'U': 378683.9401182525,
 'S': 1726.7154045245545,
 'A': 146.92176214179275,
 'L': 0.011573008727678398,
 'M': 0.102032,
 'C': 900.2905743515173,
 'CVMASS': 762.780273201937,
 'Q': 1,
 'Z': 0.910267690403166,
 'V': 1.0750853518667919e-05}

Die einzelnen Zustandsgößen lassen sich als Eigenschaften des Punktes in der Form
`Punkt.Eigenschaft` abrufen, z.B.

In [17]:
p_1.T # Temperatur des Punktes p_1 in K

273.8220637378023

Das Ergebnis ist die zu diesem Punkt gehörende Temperatur von etwa 1°C, genauer:

In [18]:
p_1.T-T_0 # Temperatur des Punktes p_1 in °C

0.6720637378023184

In Jupyter Notebooks ist es häufig übersichtlicher, mit Größen in den technisch benutzten Einheiten arbeiten zu können. Temperaturen werden z.B. in Deutschland in der Regel in °C angegeben statt in K, Enthalpien in kJ/kg statt in J/kg.

## Berücksichtigung von Einheiten

Wird beim Aufruf von `fluid_factory` der Parameter `with_units=True` angegeben, so erwartet die erzeugte Klasse die Eingabe von einheitenbehafteten Größen und gibt die Ergebnisse einheitenbehaftet aus:

In [19]:
p_1_wu = R_134a_wu(P=Q_(3,'bar'),Q=1,name='P_1_wu')
for k,v in p_1_wu.args.items():
    print(f'{k:>8} = {v:~P}')

       P = 300000.0 Pa
   Pcrit = 4059280.0 Pa
       T = 273.8220637378023 K
   Tcrit = 374.21 K
       D = 14.77016899135526 kg/m³
       H = 398995.14983938605 J/kg
       U = 378683.9401182525 J/kg
       S = 1726.7154045245545 J/K/kg
       A = 146.92176214179275 m/s
       L = 0.011573008727678398 W/K/m
       M = 0.102032 kg/mol
       C = 900.2905743515173 J/K/kg
  CVMASS = 762.780273201937 J/K/kg
       Q = 1
       Z = 0.910267690403166
       V = 1.0750853518667919×10⁻⁵ Pa·s


Für Temperatur `T` und spezifischer Enthalpie `H` ergibt sich dann beispielsweise:

In [20]:
p_1_wu.T.to('degC'), p_1_wu.H.to('kJ/kg')

(0.6720637378023184 <Unit('degree_Celsius')>,
 398.99514983938604 <Unit('kilojoule / kilogram')>)

Mit den prettyprint-Methoden von `pint` erhält man auch:

In [21]:
print(f"T = {p_1_wu.T.to('degC').round(1):~P}")
print(f"h = {p_1_wu.H.to('kJ/kg').round(2):~P}")

T = 0.7 °C
h = 399.0 kJ/kg


## Feuchte Luft

Feuchte Luft stellt einen Spezialfall dar, der durch die Funktion 

`CoolProp.CoolProp.HAPropsSi`

abgedeckt wird. Es handelt sich um die Mischung von Luft und Wasserdampf. In Anwendungen der Heizungs-, Lüftungs- und Klimatechnik spielt feuchte Luft eine große Rolle.

Wird die Funktion 

`fluid_factory('HumidAir',with_units=False,P_default=p_amb)` 

nur mit dem erforderlichen Parameter `'HumidAir'` aufgerufen, so wird eine Klasse ohne Einheiten mit dem Defaultdruck `P_default = p_amb = 101325 # Pa` initialisiert. Durch Angabe eines Wertes für `P_default` kann hier ein anderer Wert eingestellt werden.

Häufig werden Berechnungen mit feuchter Luft für einen bestimmten Druck `P = P_default` durchgeführt. Dieser Defaultdruck muss im Aufruf der Klasse nicht aufgeführt werden.

Die folgenden Zustandsgrößen werden zurückgegeben:

Größe    | Einheit  | Bedeutung
---------|----------|----------
       P |       Pa | Gesamtdruck
       W |          | Wassergehalt, absolute Luftfeuchte
       T |        K | Temperatur
       R |          | relative Luftfeuchte
       H |     J/kg | spezifische Enthalpie
       S |   J/kg/K | spezifische Entropie
       B |        K | Feuchtkugeltemperatur (bulb point temperature)
       D |        K | Taupunkttemperatur (dew point temperature)
     Vda |  m**3/kg | spezifisches Volumen des trockenluftanteils der feuchten Luft
     Vha |  m**3/kg | spezifisches Volumen der feuchten Luft
       M |     Pa*s | Viskosität
       K |    W/m/K | Wärmeleitfähigkeit
       C |   J/kg/K | spezifische Wärmekapazität bei p=const
     P_w |       Pa | Wasserdampfpartialdruck
   psi_w |          | Molenbruch des Wassers

In [22]:
HA = fluid_factory('HumidAir')
# oder
HA_wu = fluid_factory('HumidAir',with_units=True)

Ein konkreter Zustandspunkt feuchter Luft ist eine Instanz der Klasse `HA` (oder `HA_wu`). Er wird durch einen Aufruf der Form

`punkt = HA(Größe_1=Wert_1,Größe_2=Wert2,P=Druck,name=name)`

festgelegt. In diesem Aufruf sind `Größe_1=Wert1` und `Größe_2=Wert2` obligatorische Parameter. Wird der Parameter `P=Druck` weggelassen, so wird Defaultdruck der Klasse eingesetzt, der bei der Initialisierung der Klasse angegeben wurde.

Die Parameter `Größe_1` und `Größe_2` müssen voneinander unabhängig sein, um einen Wert eindeutig zu bestimmen. Nicht alle denkbaren Größen lassen sich miteinader kombinieren. So ist es zum Beispiel nicht möglich, einen Zustandspunkt durch Angabe der absoluten Luftfeuchte `W` und des Wasserdamfpartialdrucks `P_w` zu definieren.

Als generelle Regel gilt: Führen die Werte `Größe_1=Wert1` und `Größe_2=Wert2` auf einen eindeutig definierten Punkt im hx-Diagramm, so kann dieser Punkt auch für die Klasse definiert werden.

In [23]:
p_ra = HA(
    T=T_0+20, # Temperatur von 20°C in K
    R=50e-2, # relative Luftfeuchte 50%
    name='P_ra', # Bezeichnung des Punktes
)

In [24]:
p_ra.P # Defaultdruck, da beim Aufruf kein Druck eingegeben wurde

101325

In [25]:
p_ra.args # alle registrierten Zustandsgrößen

{'P': 101325,
 'W': 0.007293697701992549,
 'T': 293.15,
 'R': 0.5,
 'H': 38622.83892391293,
 'S': 139.97016819060187,
 'B': 286.92646885824075,
 'D': 282.42442581534783,
 'Vda': 0.8398598461778416,
 'Vha': 0.8337785177191823,
 'M': 1.814316044123345e-05,
 'K': 0.025866132503697774,
 'C': 1019.8524499561333,
 'P_w': 1174.488985425371,
 'psi_w': 0.011591305062179829}

In [26]:
p_au = HA(
    T=T_0+32, # Temperatur von 32°C in K
    W=12.5e-3, # absolute Luftfeuchte von 12.5 g/kg
    P=95000, # Gesamtdruck in Pa
    name='P_au', # Bezeichnung des Punktes
)

In [27]:
p_au.P

95000

In [28]:
p_au.args # alle registrierten Zustandsgrößen

{'P': 95000,
 'W': 0.0125,
 'T': 305.15,
 'R': 0.3915860024997451,
 'H': 64201.6232700863,
 'S': 246.28275870267447,
 'B': 294.37165339135277,
 'D': 289.53847417616043,
 'Vda': 0.9402603926054235,
 'Vha': 0.9286522396102949,
 'M': 1.866272841573761e-05,
 'K': 0.026729337864609244,
 'C': 1030.027877849346,
 'P_w': 1871.71464823586,
 'psi_w': 0.019702259455114318}

Es gibt Zustandspunkte, für die nicht alle Größen berechnet werden können: Zum Beispiel ist die relative Luftfeuchte `R` nur für Luftzustände definiert, in denen der Dampfdruck kleiner als der Dampfsättigungsdruck ist. Für eine Temperatur von 20°C ist beispielsweise

In [29]:
HA_wu(T=Q_(20,'degC'),R=1).W.to('g/kg')

14.760495260665328 <Unit('gram / kilogram')>

Bei einem Wassergehalt von `W > Q_(14.76...,'g/kg'` ist `R` nicht mehr definiert:

In [30]:
try:
    pt = HA_wu(T=Q_(20,'degC'),W=Q_(15,'g/kg'))
    print(pt.R)
except Exception as e:
    print(e)

The output for key (7) with value (1.01584) is outside the range of validity: (0) to (1) :: inputs were:"R","T",2.9314999999999998e+02,"W",1.4999999999999999e-02,"P",1.0132500000000000e+05 


Entsprechend führt in diesem Fall auch der Aufruf `pt.args` zu einem Fehler: 

In [31]:
try:
    pt.args
except Exception as e:
    print(e)

The output for key (7) with value (1.01584) is outside the range of validity: (0) to (1) :: inputs were:"R","T",2.9314999999999998e+02,"W",1.4999999999999999e-02,"P",1.0132500000000000e+05 


Andere Zustandsgrößen dieses Zustandspunktes sind allerdings noch definiert. Daher existiert die Methode `Klasse.args_or_errormessages`:

In [32]:
pt.args_or_errormessages

{'P': 101325.0 <Unit('pascal')>,
 'W': 0.015 <Unit('dimensionless')>,
 'T': 293.15 <Unit('kelvin')>,
 'R': ValueError('The output for key (7) with value (1.01584) is outside the range of validity: (0) to (1) :: inputs were:"R","T",2.9314999999999998e+02,"W",1.4999999999999999e-02,"P",1.0132500000000000e+05 '),
 'H': 58166.431230093214 <Unit('joule / kilogram')>,
 'S': 207.78828664668217 <Unit('joule / kelvin / kilogram')>,
 'B': 293.3257073663906 <Unit('kelvin')>,
 'D': 293.4039498500014 <Unit('kelvin')>,
 'Vda': 0.8501068744940903 <Unit('meter ** 3 / kilogram')>,
 'Vha': 0.8375437187133894 <Unit('meter ** 3 / kilogram')>,
 'M': 1.8078509265290083e-05 <Unit('pascal * second')>,
 'K': 0.025858022919239887 <Unit('watt / kelvin / meter')>,
 'C': 1034.5556663027326 <Unit('joule / kelvin / kilogram')>,
 'P_w': 2386.1950403881024 <Unit('pascal')>,
 'psi_w': 0.023549914042813744 <Unit('dimensionless')>}